In [ ]:
#for dev purpose
import getpass
import os
USERNAME = os.getenv('USERNAME').lower()
new_pull = True

In [ ]:
# import python packages
import os
import sys
if sys.version_info[1]==12:
    os.add_dll_directory(r'C:/Program Files/Python/312/Lib/site-packages/clidriver/bin')
else:
    pass
import pandas as pd
import datetime as dt
import time
import warnings
from snp_query_box import DsnpHelperFunction, hraQueries
from snp_query_box.dsnp_transform import hraResultTransform
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
print(f"User: {USERNAME} Started creating sdoh_status_from_hra_df in the shared storage")

In [ ]:
# set the starting time
start_time = time.time()

In [ ]:
# calculate the dates
today = dt.datetime.today()
today_str = today.strftime('%Y-%m-%d')
last_date_of_last_month_str = DsnpHelperFunction.last_date_of_last_month(today_str, output_type="string")
first_date_of_last_month = DsnpHelperFunction.first_date_of_last_month(today_str)
last_day_reporting_year = DsnpHelperFunction.last_date_of_year(first_date_of_last_month)
reporting_end_date = last_date_of_last_month_str
print(reporting_end_date)

In [ ]:
# set the storage and output path
storage_path = r'//mbip/medicarepBI/Projects/COE/DSNP/dsnp_data_storage/monthly_flat_files'
output_path = f'{storage_path}/{reporting_end_date}'

isExist = os.path.exists(output_path)
print(output_path)
if not isExist:
    # Create a new directory because it does not exist
     os.makedirs(output_path)
     print("The new directory is created!")
else:
      print("The folder already exist")
print("Output files will sit in the folder above.")

In [ ]:
#locate data dump folder
if new_pull:
    pull_hra_qa_by_section_services_and_support = hraQueries.pull_hra_qa_by_section_bq(asad_section_nm = 'Services and Support')
    pull_hra_qa_by_section_services_and_support.to_parquet(f"{output_path}/pull_hra_qa_by_section_services_and_support.parquet")
else:
    try:
        pull_hra_qa_by_section_services_and_support = pd.read_parquet(f"{output_path}/pull_hra_qa_by_section_services_and_support.parquet")

    except:
        pull_hra_qa_by_section_services_and_support = hraQueries.pull_hra_qa_by_section_bq(asad_section_nm = 'Services and Support')
        pull_hra_qa_by_section_services_and_support.to_parquet(f"{output_path}/pull_hra_qa_by_section_services_and_support.parquet")

In [ ]:
hra_service_and_support = (pull_hra_qa_by_section_services_and_support.pipe(hraResultTransform.get_most_recent_hra)  #possible template 3 hrt.version_three_to_four
        .pipe(hraResultTransform.pivot_hra_qa)
        .pipe(hraResultTransform.clean_service_support))

In [ ]:
hra_service_and_support.head(2)

In [ ]:
hra_service_and_support[hra_service_and_support['dsnp_member_id']=='101068528400']

In [ ]:
unique_elements = set([item for sublist in hra_service_and_support["any_concern"] for item in sublist if item != ''])
for element in unique_elements:
    hra_service_and_support['SDOH_' + element.capitalize()] = hra_service_and_support['any_concern'].apply(lambda x: 'Y' if element in x else 'N')

In [ ]:
hra_service_and_support.rename(columns = {"SDOH_Medicine or any health care (medical, dental, mental health, vision)":
                                                     "SDOH_Medicine or any health care"}, inplace=True)

In [ ]:
sdoh_columns = ["SDOH_Food",
                "SDOH_Housing",
                "SDOH_Utilities",
                "SDOH_Medicine or any health care",
                "SDOH_Child care",
                "SDOH_Clothing",
                "SDOH_Transportation",
                "SDOH_Phone"]
hra_service_and_support[sdoh_columns + ["dsnp_member_id", "any_concern"]]

In [ ]:
hra_service_and_support[hra_service_and_support['dsnp_member_id']=='101773006400']

In [ ]:
hra_service_and_support.to_parquet(f'{output_path}/sdoh_status_from_hra_df.parquet')

In [ ]:
#def clean_invalid_utf8(value):
#    if isinstance(value, str):
#        value=re.sub(r'�', '', value)
#    return value
#
#rows = []
#for row in query_job.result():
#    for column, value in row.items():
#        if '�' in value:
#            print(value)
#            cleaned_row = {column:clean_invalid_utf8(value)}
#            print(cleaned_row)
#        else:
#            cleaned_row = {column:value}
#    
#    rows.append(cleaned_row)
#test_df = pd.DataFrame(rows)
#
#test_df = query_job.to_dataframe()